## DecisionTree

In [1]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('train_std.csv')
df_test = pd.read_csv('test_std.csv')

In [2]:
df_train.head()

,id,Jenis Kelamin,Gaji,Berpisah,Cerai,Janda,Menikah,Menikah LDR,Ekesekutif Managerial,Mesin Inspeksi,...,SD,SMA,Sarjana,Sekolah Professional,Jam per Minggu,Jmlh Tahun Pendidikan,Umur,Berat Akhir,Keuntungan Kapital,Kerugian Capital
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,-0.439516,-0.419405,-1.291639,0.500754,-0.146026,-0.21628
1,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,-0.035226,-1.584555,0.755732,-0.464550,-0.146026,-0.21628
2,2,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0.369063,0.745744,0.390130,-0.660437,0.450185,-0.21628
3,3,1,0,0,0,0,0,0,0,0,...,0,0,1,0,-0.439516,1.134128,-1.072278,0.044406,-0.146026,-0.21628
4,4,1,1,0,0,0,1,0,0,0,...,0,0,0,0,-0.035226,1.522511,-0.414194,0.280595,1.896261,-0.21628


In [3]:
x_train = df_train.drop(['id','Gaji'], axis=1)
y_train = df_train['Gaji']

## MODEL

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [27]:
params_dtc = {'criterion':['gini','entropy'],'max_depth':np.arange(1,50,1), 'min_samples_split':np.arange(2,10), 'min_samples_leaf':np.arange(1,10,1),}
model_dtc = DecisionTreeClassifier()

In [28]:
gs = GridSearchCV(model_dtc, params_dtc, scoring='roc_auc', n_jobs=-1, cv=2, verbose=2)
gs.fit(x_train, y_train)

Fitting 2 folds for each of 7056 candidates, totalling 14112 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 6837 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 7930 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 9105 tasks      | 

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='roc_auc', verbose=2)

In [30]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_leaf': 9,
 'min_samples_split': 8}

In [31]:
gs.best_score_

0.9020634438969048

In [22]:
model_dtc = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=9, min_samples_split=3, max_leaf_nodes=None)

In [23]:
model_dtc.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=9, min_samples_split=3)

## PREDICT

In [24]:
x_test = df_test.drop('id',axis=1)

In [25]:
temp = model_dtc.predict(x_test)

In [26]:
df_pred = pd.DataFrame({'Gaji':temp,})

In [27]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9599 entries, 0 to 9598
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Gaji    9599 non-null   int64
dtypes: int64(1)
memory usage: 75.1 KB


In [28]:
df_pred['Gaji'].value_counts()

0    7890
1    1709
Name: Gaji, dtype: int64

## EVAL

In [29]:
from sklearn.model_selection import cross_validate

In [30]:
cv = cross_validate(model_dtc, x_test, y=df_pred, scoring='roc_auc', cv=15)

In [31]:
cv['test_score'].mean()

0.9923732656506393

## SAVING

In [32]:
df_pred = pd.DataFrame({'id':df_test['id'], 'Gaji':temp})

In [34]:
df_pred

,id,Gaji
0,35994,0
1,35995,0
2,35996,0
3,35997,0
4,35998,0
...,...,...
9594,45588,0
9595,45589,0
9596,45590,0
9597,45591,0


In [35]:
df_pred.to_csv('Gaji_DTC_gini_std_bfill_naufalhisyam46.csv', index=False)

In [36]:
df_pred

,id,Gaji
0,35994,0
1,35995,0
2,35996,0
3,35997,0
4,35998,0
...,...,...
9594,45588,0
9595,45589,0
9596,45590,0
9597,45591,0
